# Exercise: Creating the Bob In-Flight HTLC Commitment Transaction

Now as exercise let's create the Bob In-Flight Commitment Transaction.

## The Unsigned Transaction

### The Input

The input is the same.

### The Outputs

Bob transaction will have 4 outputs also:
* local anchor output
* remote anchor output
* accepted htlc output
* to remote output

#### The Accepted HTLC Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +----------------------------------------------------+                        |
      | P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP     |                        |
      |                                                    |                        |
      | cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP         |                        |
      +----------------------------------------------------+                        |
      +-------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY |
      |                                                                               |
      | P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG                    |
      +-------------------------------------------------------------------------------+


In [ ]:
# Create Alice Local Public Key which in this case is the P(remote) for Bob
# Use the method 'get_pubkey' from the basepoint opject (alice payment) and the alice per commitment point to derive the alice payment pubkey
alice_payment_pubkey = alice_payment.get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Payment PubKey: {alice_payment_pubkey.get_bytes(bip340=False).hex()}")

# Create Bob Delayed Public Key
bob_delayed_pubkey = derivate_key(bob_node_seed, family=4, channel_index=0).get_pubkey(bob_per_commitment.get_pub())
print(f"Bob Delayed PubKey: {bob_delayed_pubkey.get_bytes(bip340=False).hex()}")

# Create Alice Revocation Public Key
alice_revocation_pubkey = derivate_revocation_key(alice_node_seed, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Revocation PubKey: {alice_revocation_pubkey.get_bytes(bip340=False).hex()}")

# Outputs for Bob In-Flight Commitment Transaction
# 0x04 outputs
output_count = bytes.fromhex("04")

# ANCHOR OUTPUTS
# Anchor output amount (we use the same amount as before)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapBranch", hash_input)

# Alice Anchor Output Tagged Hash (remote anchor for Bob)
taptweak = tagged_hash("TapTweak", alice_payment_pubkey.get_bytes() + script_root)
alice_payment_pubkey_tweaked = alice_payment_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_payment_pubkey_tweaked.get_bytes()) + alice_payment_pubkey_tweaked.get_bytes()

# Bob Anchor Output Tagged Hash (local anchor for Bob)
taptweak = tagged_hash("TapTweak", bob_delayed_pubkey.get_bytes() + script_root)
bob_delayed_pubkey_tweaked = bob_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_anchor_spk = bytes.fromhex("51") + varint_len(bob_delayed_pubkey_tweaked.get_bytes()) + bob_delayed_pubkey_tweaked.get_bytes()

# Sort by scriptPubKey (lexicographic order)
anchors = [
    (alice_anchor_spk, "alice_anchor"),
    (bob_anchor_spk,   "bob_anchor"),
]

anchors_sorted = sorted(anchors, key=lambda x: x[0])

# ACCEPTED HTLC OUTPUT
# HTLC output amount (we use the same amount as before)

# Create the leaf scripts A and B
# script A: P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP
# Add cltv_delta (40 blocks) to current block height
cltv_expiry = node.getblockcount() + 40
scriptA = CScript([alice_htlc_pubkey.get_bytes(bip340=False), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, cltv_expiry, OP_CHECKLOCKTIMEVERIFY, OP_DROP])

# Leaf script A: OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
# We use the same dummy payment hash as before.
scriptB = CScript([OP_SIZE, bytes.fromhex("20"), OP_EQUALVERIFY, OP_HASH160, payment_hash, OP_EQUALVERIFY, bob_htlc_pubkey.get_bytes(bip340=False), OP_CHECKSIGVERIFY, alice_htlc_pubkey.get_bytes(bip340=False), OP_CHECKSIG])

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)

# Use the tagged_hash function to compute TapLeaf A and B
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# Compute Internal node TapBranch AB
root_ab = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# Compute TapTweak
taptweak = tagged_hash("TapTweak", alice_revocation_pubkey.get_bytes() + root_ab)

# Tweak the alice revocation pubkey
alice_revocation_pubkey_tweaked = alice_revocation_pubkey.tweak_add(taptweak)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_accepted_htlc_spk = bytes.fromhex("51") + varint_len(alice_revocation_pubkey_tweaked.get_bytes()) + alice_revocation_pubkey_tweaked.get_bytes()

# TO REMOTE OUTPUT
# to remote output amount
# As we have no to_local output in this case, the to_remote value pay the fees
# Using the same fee as before
to_remote_value_sat =  channel_value_sat - anchor_output_value_satoshis * 2 - htlc_output_value_satoshis - commitment_fee
to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)
print("channel_value_sat, anchor_output_value_satoshis * 2, commitment_fee: ", channel_value_sat, anchor_output_value_satoshis, commitment_fee)

# P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# Use CScript to create the to remote script
script = CScript([alice_payment_pubkey.get_bytes(), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# To Remote Output script_root
# Use the tagged_hash function to compute the script root
script_root = tagged_hash("TapLeaf", hash_input)

# To Remote Output Tagged Hash
# Use the tagged_hash function to compute the taptweak
taptweak = tagged_hash("TapTweak", NUMS.get_bytes() + script_root)
# Use the tweak_add to tweak the NUMS key
NUMS_tweaked = NUMS.tweak_add(taptweak)
# Create the scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_to_remote_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes()) + NUMS_tweaked.get_bytes()

outputs = (
    anchor_output_value
    + varint_len(anchors_sorted[0][0])
    + anchors_sorted[0][0]
    + anchor_output_value
    + varint_len(anchors_sorted[1][0])
    + anchors_sorted[1][0]
    + to_remote_value
    + varint_len(bob_to_remote_spk)
    + bob_to_remote_spk
    + htlc_output_value
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
)

# Locktime is the same

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))


## The sighash for the key path spend

In [ ]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()


## Signing the sighash

In [ ]:
# Reusing the reordered participants to match the sorted pubkeys

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for bob to get deterministic results and nonce_gen for Alice jit nonce
    if pk == pubkey_bob_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
# Use nonce_agg to aggregate the pubnonces
agg_nonce = nonce_agg(pubnonces)
# Use the SessionContext to create the gignature session
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
# Use sign function to produce the partial signatures
psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
# Use the function partial_sig_agg to aggregate the partial signatures
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


## The signed transaction

In [ ]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_tx.hex()]))